<a href="https://colab.research.google.com/github/microsoft/autogen/blob/main/notebook/agentchat_stream.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Interactive LLM Agent Dealing with Data Stream

AutoGen offers conversable agents powered by LLM, tool, or human, which can be used to perform tasks collectively via automated chat. This framework allows tool use and human participation through multi-agent conversation.
Please find documentation about this feature [here](https://microsoft.github.io/autogen/docs/Use-Cases/agent_chat).

In this notebook, we demonstrate how to use customized agents to continuously acquire news from the web and ask for investment suggestions.

## Requirements

AutoGen requires `Python>=3.8`. To run this notebook example, please install:
```bash
pip install pyautogen
```

In [ ]:
# %pip install pyautogen~=0.1.0

In [11]:
import pandas as pd

csvFilePath = "SamplesData/Orders.csv"

df = pd.read_csv(csvFilePath,sep=',')

df.to_json(orient="records", force_ascii=False)

# df

'[{"OrderId":"4f2afbc8507e4effb17e81dae7a4fe19","OrderNo":1,"CustomerNo":"customer1","Product":"烤白薯","Price":0.5},{"OrderId":"1aaa41cd9ddf46fa93aeea0478992746","OrderNo":2,"CustomerNo":"customer2","Product":"酱香拿铁","Price":3.1},{"OrderId":"2187eb0eedcf437bbc4cb310f51e9ed9","OrderNo":3,"CustomerNo":"customer3","Product":"蒜香卡布奇诺","Price":10.1},{"OrderId":"05e94bc32e9d446980b50d65d85e32a3","OrderNo":4,"CustomerNo":"customer4","Product":"烤串","Price":0.1},{"OrderId":"6d70527d4a1144f889a259b778ea9699","OrderNo":5,"CustomerNo":"customer5","Product":"蒜香卡布奇诺","Price":10.1},{"OrderId":"7e50da85864340a385a7405449271362","OrderNo":6,"CustomerNo":"customer6","Product":"蒜香卡布奇诺","Price":10.1},{"OrderId":"e76f18532242484c81eb339a4f3655ef","OrderNo":7,"CustomerNo":"customer7","Product":"蒜香卡布奇诺","Price":10.1},{"OrderId":"e7326fa0e2d240699b54a578d1ad7809","OrderNo":8,"CustomerNo":"customer8","Product":"肉夹馍","Price":1.1},{"OrderId":"4b28e757a19d41c5b5944e0cf53765d8","OrderNo":9,"CustomerNo":"customer9","Pr

## Set your API Endpoint

The [`config_list_from_json`](https://microsoft.github.io/autogen/docs/reference/oai/openai_utils#config_list_from_json) function loads a list of configurations from an environment variable or a json file.


In [1]:
import autogen

config_list = autogen.config_list_from_json("OAI_CONFIG_LIST")


config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        # "model": ["gpt-4", "gpt-4-0314", "gpt4", "gpt-4-32k", "gpt-4-32k-0314", "gpt-4-32k-v0314"],
        "model": ["gpt-35-turbo-16k-dep-001"]
    },
)

/home/awesomeyuer/miniconda3/envs/autogen-py311-env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


It first looks for environment variable "OAI_CONFIG_LIST" which needs to be a valid json string. If that variable is not found, it then looks for a json file named "OAI_CONFIG_LIST". It filters the configs by models (you can filter by other keys as well). Only the models with matching names are kept in the list based on the filter condition.

The config list looks like the following:
```python
config_list = [
    {
        'model': 'gpt-4',
        'api_key': '<your OpenAI API key here>',
    },  # OpenAI API endpoint for gpt-4
    {
        'model': 'gpt-4',
        'api_key': '<your first Azure OpenAI API key here>',
        'api_base': '<your first Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2023-06-01-preview',
    },  # Azure OpenAI API endpoint for gpt-4
    {
        'model': 'gpt-4',
        'api_key': '<your second Azure OpenAI API key here>',
        'api_base': '<your second Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2023-06-01-preview',
    },  # another Azure OpenAI API endpoint for gpt-4
    {
        'model': 'gpt-3.5-turbo',
        'api_key': '<your OpenAI API key here>',
    },  # OpenAI API endpoint for gpt-3.5-turbo
    {
        'model': 'gpt-3.5-turbo',
        'api_key': '<your first Azure OpenAI API key here>',
        'api_base': '<your first Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2023-06-01-preview',
    },  # Azure OpenAI API endpoint for gpt-3.5-turbo
    {
        'model': 'gpt-3.5-turbo',
        'api_key': '<your second Azure OpenAI API key here>',
        'api_base': '<your second Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2023-06-01-preview',
    },  # another Azure OpenAI API endpoint for gpt-3.5-turbo
]
```

If you open this notebook in colab, you can upload your files by clicking the file icon on the left panel and then choosing "upload file" icon.

You can set the value of config_list in other ways you prefer, e.g., loading from a YAML file.

## Example Task: Investment suggestion with realtime data

We consider a scenario where news data are streamed from a source, and we use an assistant agent to provide investment suggestions based on the data continually.

First, we use the following code to simulate the data stream process.

In [2]:
import asyncio
import AlphaVantageApiKey

def get_market_news(ind, ind_upper):
    import requests
    import json

    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    url = f'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers=AAPL&sort=LATEST&limit=5&apikey={AlphaVantageApiKey.ApiKey}'
    r = requests.get(url)
    data = r.json()
    # with open('market_news_local.json', 'r') as file:
    #     # Load JSON data from file
    #     data = json.load(file)
    data = {
        "feed": [
            {
                "title": "Palantir CEO Says Our Generation's Atomic Bomb Could Be AI Weapon - And Arrive Sooner Than You Think - Palantir Technologies  ( NYSE:PLTR ) ",
                "summary": "Christopher Nolan's blockbuster movie \"Oppenheimer\" has reignited the public discourse surrounding the United States' use of an atomic bomb on Japan at the end of World War II.",
                "overall_sentiment_score": 0.009687,
            },
            {
                "title": '3 "Hedge Fund Hotels" Pulling into Support',
                "summary": "Institutional quality stocks have several benefits including high-liquidity, low beta, and a long runway. Strategist Andrew Rocco breaks down what investors should look for and pitches 3 ideas.",
                "banner_image": "https://staticx-tuner.zacks.com/images/articles/main/92/87.jpg",
                "overall_sentiment_score": 0.219747,
            },
            {
                "title": "PDFgear, Bringing a Completely-Free PDF Text Editing Feature",
                "summary": "LOS ANGELES, July 26, 2023 /PRNewswire/ -- PDFgear, a leading provider of PDF solutions, announced a piece of exciting news for everyone who works extensively with PDF documents.",
                "overall_sentiment_score": 0.360071,
            },
            {
                "title": "Researchers Pitch 'Immunizing' Images Against Deepfake Manipulation",
                "summary": "A team at MIT says injecting tiny disruptive bits of code can cause distorted deepfake images.",
                "overall_sentiment_score": -0.026894,
            },
            {
                "title": "Nvidia wins again - plus two more takeaways from this week's mega-cap earnings",
                "summary": "We made some key conclusions combing through quarterly results for Microsoft and Alphabet and listening to their conference calls with investors.",
                "overall_sentiment_score": 0.235177,
            },
        ]
    }
    feeds = data["feed"][ind:ind_upper]
    feeds_summary = "\n".join(
        [
            f"News summary: {f['title']}. {f['summary']} overall_sentiment_score: {f['overall_sentiment_score']}"
            for f in feeds
        ]
    )
    return feeds_summary

data = asyncio.Future()

async def add_stock_price_data():
    # simulating the data stream
    for i in range(0, 5, 1):
        latest_news = get_market_news(i, i + 1)
        if data.done():
            data.result().append(latest_news)
        else:
            data.set_result([latest_news])
        # print(data.result())
        await asyncio.sleep(5)

data_task = asyncio.create_task(add_stock_price_data())


Then, we construct agents. An assistant agent is created to answer the question using LLM. A UserProxyAgent is created to ask questions, and add the new data in the conversation when they are available.

In [3]:
import autogen

# create an AssistantAgent instance named "assistant"
assistant = autogen.AssistantAgent(
    name="assistant",
    llm_config={
        "request_timeout": 600,
        "seed": 41,
        "config_list": config_list,
        "temperature": 0,
    },
    system_message="You are a financial expert.",
)
# create a UserProxyAgent instance named "user"
user_proxy = autogen.UserProxyAgent(
    name="user",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=5,
    code_execution_config=False,
    default_auto_reply=None,
)

async def add_data_reply(recipient, messages, sender, config):
    await asyncio.sleep(0.1)
    data = config["news_stream"]
    if data.done():
        result = data.result()
        if result:
            news_str = "\n".join(result)
            result.clear()
            return (
                True,
                f"Just got some latest market news. Merge your new suggestion with previous ones.\n{news_str}",
            )
        return False, None

user_proxy.register_reply(autogen.AssistantAgent, add_data_reply, 1, config={"news_stream": data})

We invoke the `a_initiate_chat()` method of the user proxy agent to start the conversation.

In [4]:
await user_proxy.a_initiate_chat(
    assistant,
    message="""Give me investment suggestion in 3 bullet points.""",
)
while not data_task.done() and not data_task.cancelled():
    reply = await user_proxy.a_generate_reply(sender=assistant)
    if reply is not None:
        await user_proxy.a_send(reply, assistant)

user (to assistant):

Give me investment suggestion in 3 bullet points.

--------------------------------------------------------------------------------
assistant (to user):

1. Diversify your portfolio: Invest in a mix of different asset classes such as stocks, bonds, real estate, and commodities. This will help spread your risk and potentially increase your returns.

2. Consider long-term investments: Look for opportunities that have the potential for growth over the long term. This could include investing in emerging markets, technology companies, or sustainable industries.

3. Stay informed and seek professional advice: Keep up-to-date with market trends and economic news to make informed investment decisions. Additionally, consider consulting with a financial advisor who can provide personalized guidance based on your financial goals and risk tolerance.

--------------------------------------------------------------------------------
user (to assistant):

Just got some latest mar